# 구글포토 이미지 OCR 적용후 문제유형
1. 구글포토 앨범이름으로 이미지 읽기
2. 구글 vision API 로 이미지 OCR 적용
3. IMAGE SHOW


# 구글 API 인증키 설정
1. google vision API 사용설정
구글클라우드 콘솔에서 vision API 사용설정후 인증키 json 파일 다운로드
현재 client_secret_desktop.json 파일로 사용
2. google photo 사용설정
구글포토 인증키 service-account-file.json 파일로 사용

In [1]:
%pip install --upgrade google-api-python-client
%pip install --upgrade google-cloud-vision
%pip install --upgrade google-auth-oauthlib

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviou

### 앨범목록 출력, 이미지 OCR 적용후 적용된 이미지 block 출력


In [4]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
from google.cloud import vision
from PIL import Image, ImageDraw
import requests
from enum import Enum

# 구글 포토 API를 사용하여 앨범에서 사진을 가져오는 파이썬 예제를 제공하겠습니다. Google Photos API를 사용하기 전에 Google Cloud Console에서 프로젝트를 생성하고,
# Google Photos Library API를 활성화하며, OAuth 2.0 클라이언트 ID를 생성해야 합니다. 이 예제는 이러한 설정이 이미 완료되었다고 가정합니다.
#
# 사전 준비사항
# Google Photos API에 액세스하기 위한 클라이언트 ID와 클라이언트 비밀을 준비합니다.
# google-auth-oauthlib와 google-auth-httplib2 라이브러리를 사용합니다. 필요한 라이브러리가 없다면,
# pip install google-auth-oauthlib google-auth-httplib2를 실행하여 설치해주세요.

# OAuth 2.0 클라이언트 ID와 비밀 정보 설정
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5


def draw_boxes(image, bounds, color):
    """Draws a border around the image using the hints in the vector list.

    Args:
        image: the input image object.
        bounds: list of coordinates for the boxes.
        color: the color of the box.

    Returns:
        An image with colored bounds added.
    """
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x,
                bound.vertices[0].y,
                bound.vertices[1].x,
                bound.vertices[1].y,
                bound.vertices[2].x,
                bound.vertices[2].y,
                bound.vertices[3].x,
                bound.vertices[3].y,
            ],
            None,
            color,
        )
    return image


def process_word(word, feature):
    bounds = []
    if feature == FeatureType.WORD:
        bounds.append(word.bounding_box)

        if feature == FeatureType.SYMBOL:
            bounds.extend(symbol.bounding_box for symbol in word.symbols)
    return bounds


def process_paragraph(paragraph, feature):
    bounds = []
    if feature == FeatureType.PARA:
        bounds.append(paragraph.bounding_box)

    for word in paragraph.words:
        bounds.extend(process_word(word, feature))
    return bounds


def get_document_bounds(image_file, feature):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service-account-file.json'
    client_options = {'api_endpoint': 'us-vision.googleapis.com'}
    client = vision.ImageAnnotatorClient(client_options=client_options)

    with open(image_file, "rb") as image_file:
        content = image_file.read()
    content = {'content': content}
    image = vision.Image(**content)
    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    bounds = []

    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                bounds.extend(process_paragraph(paragraph, feature))
    return bounds


def save_txt(filein):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service-account-file.json'

    client_options = {'api_endpoint': 'us-vision.googleapis.com'}
    client = vision.ImageAnnotatorClient(client_options=client_options)

    with open(filein, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    output_file_path = filein.split('/')[1].split('.')[0] + ".txt"

    with open(output_file_path, "w", encoding="utf-8") as output_file:
        pages_text = ""
        for page in document.pages:
            page_text = ""
            for block in page.blocks:
                block_text = ""
                for paragraph in block.paragraphs:
                    paragraph_text = ""
                    for word in paragraph.words:
                        word_text = "".join([symbol.text for symbol in word.symbols])
                        paragraph_text += word_text + " "

                    block_text += paragraph_text + "\n"
                    # print(block_text)
                page_text += block_text + "\n"
                # print(page_text)
            pages_text += page_text + "\n"
            # print(pages_text)

        print(pages_text)
        output_file.write(pages_text)
    print("텍스트를 파일에 작성하였습니다.")


def render_doc_text(filein, fileout):
    """Outlines document features (blocks, paragraphs and words) given an image.

    Args:
        filein: path to the input image.
        fileout: path to the output image.
    """

    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, "blue")
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, "red")
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, "yellow")

    save_txt(filein)

    if fileout != 0:
        image.save(fileout)
    else:
        image.show()


# OAuth 2.0 클라이언트 ID와 비밀 정보 설정
client_secrets_file = 'client_secret_desktop.json'

# 스코프 설정
# https://www.googleapis.com/auth/photoslibrary.readonly 스코프를 사용하여 읽기 전용 액세스 권한을 요청합니다.
scopes = ['https://www.googleapis.com/auth/photoslibrary.readonly']

# 사용자 인증 흐름 설정
flow = InstalledAppFlow.from_client_secrets_file(
    client_secrets_file,
    scopes=scopes
)

# 사용자 인증 및 액세스 토큰 획득
# credentials = flow.run_console()
credentials = flow.run_local_server(port=8088)
# http://127.0.0.1/accounts/google/login/callback/
# credentials = flow.from_client_secrets_file(client_secrets_file)
# Google Photos API 서비스 객체 생성
service = build('photoslibrary', 'v1', credentials=credentials, static_discovery=False)
# ..http=self.credentials.authorize(Http()),static_discovery=False)
# 앨범 목록 가져오기
results = service.albums().list(
    pageSize=10,  # 한 페이지에 표시할 앨범 수
    excludeNonAppCreatedData=False  # 앱 외부에서 생성된 데이터 포함 여부
).execute()

albums = results.get('albums', [])

if not albums:
    print('앨범을 찾을 수 없습니다.')
else:
    print('앨범 목록:')
    for album in albums:
        print(f"{album['title']} (ID: {album['id']})")

# 특정 앨범에서 사진 가져오기 (첫 번째 앨범의 ID 사용)
if albums:
    album_id = 'AIqQLL9Z5GMPBtXlB0nnw--TOv1_mZ23XeL1ExlGLt16Iwh-MyApT3UMfRSYhCKDhRjVmT0fwNUW'  # albums[0]['id'] # 첫 번째 앨범의 ID

    response = service.mediaItems().search(body={'albumId': album_id}).execute()
    items = response.get('mediaItems', [])
    if not items:
        print('사진을 찾을 수 없습니다.')
    else:
        for item in items:
            if item['filename'] == "IMG_7629.png":
                print(f"사진 제목: {item['filename']}, URL: {item['baseUrl']}")
                # 이미지 URL 설정
                image_url = item['baseUrl']
                original_image = '=w0-h0-n-k-no'
                image_url = image_url + original_image

                # 이미지 다운로드
                response = requests.get(image_url)
                if response.status_code == 200:
                    with open('download/' + item['filename'], "wb") as image_file:
                        image_file.write(response.content)
                    print("이미지 다운로드 완료")

                    render_doc_text('download/' + item['filename'], 0)

                else:
                    print("이미지를 다운로드하는 동안 문제가 발생했습니다. 상태 코드:", response.status_code)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=321040457013-9c5098pkpmsdma3c2psajfong2kdv1kq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8088%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fphotoslibrary.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-vision&state=OFs1v4RnehyzYMYKTgrtpLxDIDRNsj&access_type=offline
Processing image 1/17
try JSON Object
{'questions': [{'question_no': '06', 'question': '다음 중 데이터 분석 수준 진단 결과에 대한 설명으로 옳지 않은 것은?', 'choices': ['1. 정착형은 준비도는 높으나 조직, 인력, 분석 업무, 분석 기법 등을 기업 내부에서 제한적으로 사용하는 경우이다.', '2. 준비형은 기업에 필요한 데이터, 인력, 조직, 분석 업무, 분서 기법 등이 적용되어 있지 않아 사전 준비가 필요한 경우이다.', '3. 도입형은 기업에서 활용하는 분석 업무, 기법 등이 부족하지만 적용 조직 등 준비도가 높아 바로 도입할 수 있는 경우이다.', '4. 확산형은 기업에 필요한 6가지 분석 구성 요소를 갖추고 있고, 현재 부분적으로 도입되어 지속적인 확산이 필요한 경우이다.'], 'answer': '1', 'explanation': '정착형은 데이터 분석 수준 진단 결과에 대한 설명으로 옳지 않은 내용입니다.'}, {'question_no': '07', 'question': '다음 중 분석 준비도(Readiness)의 진단

AttributeError: 'str' object has no attribute 'append'